# Research Project Topic
I will be using simulation data to determine the kinematics of disk stars in the MW and M31 merger remnant.

### Question(s) to be pursued
The first question I would like to address (because it seems to be a good starting point) is determining the velocity dispersion of the remnant as a function of radius, but in order to do this I must first determine the mean velocity of the remnant as a function of radius.

### Plot(s) to be produced
The first plot I would like to produce is the velocity dispersion of the disk stars in the remnant as a function of radius.

In [15]:
#first, import necessary modules
import numpy as np
import astropy.units as u
from astropy.constants import G
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
from Readfile import Read
from CenterOfMass import CenterOfMass
#import MassProfile

In [16]:
#since the MW and M31 merge at ~ 6.3 Gyr, pick a snapshot to look at that is a bit of time after that, say around 8 Gyr
#I chose Snapshot 575 because it's a nice number (And has a time of ~8.2Gyr)

#step one: Calculate the COM of the merger remnant
#step two: calculate the positions/velocities of all the disk particles relative to that COM
#step three: calculate the mean velocity as a function of radius

In [17]:
#execute step 1
#create a COM object for MW and M31
# Create a Center of mass object for the MW, M31
MWCOM = CenterOfMass("MW_575.txt", 2)
M31COM = CenterOfMass("M31_575.txt", 2)

# MW:   store the position and velocity COM
MW_COMP = MWCOM.COM_P(0.1, 2.0)
print("MW COM Position = ", MW_COMP)
MW_COMV = MWCOM.COM_V(MW_COMP[0], MW_COMP[1], MW_COMP[2])
print("MW COM Velocity = ", MW_COMV)

#M31: store position and velocity COM
M31_COMP = M31COM.COM_P(0.1, 2.0)
print("M31 COM Position = ", M31_COMP)
M31_COMV = M31COM.COM_V(M31_COMP[0], M31_COMP[1], M31_COMP[2])
print("M31 COM Velocity = ", M31_COMV)

#should I use the snapshot where the COMs are the closest and just use one of them? Or should I take an average and use
#that average as the merger COM?

MW COM Position =  [89.55 94.64 65.77] kpc
MW COM Velocity =  [ 37.71 -17.16  29.69] km / s
M31 COM Position =  [90.45 95.42 66.09] kpc
M31 COM Velocity =  [ 34.63 -15.84  25.97] km / s


In [31]:
#read in the data and store variables for them
time, total, MWdata = Read("MW_575.txt")
time2, total2, M31data = Read("M31_575.txt")

#store data for wanted particle type 
index = np.where(MWdata['type'] == 2)
MWx = MWdata['x'][index]
MWy = MWdata['y'][index]
MWz = MWdata['z'][index]
MWvx = MWdata['vx'][index]
MWvy = MWdata['vy'][index]
MWvz = MWdata['vz'][index]
MWmass = MWdata['m'][index]

index2 = np.where(M31data['type'] == 2)
M31x = M31data['x'][index2]
M31y = M31data['y'][index2]
M31z = M31data['z'][index2]
M31vx = M31data['vx'][index2]
M31vy = M31data['vy'][index2]
M31vz = M31data['vz'][index2]
M31mass = M31data['m'][index2]

In [32]:
#for now I will just set the COM position and velocity to be equal to that calculated for the MW
MergerCOMP = MW_COMP.value
MergerCOMV = MW_COMV.value

#execute step 2
#create a modified version of particle properties and use it to determine the properties of particles within the remnant
def ParticleInfo(filename, particletype, particlenum):
    #read in file and extract needed data from it
    time, total, data = Read(filename)
    
    #store data for wanted particle type 
    index = np.where(data['type'] == particletype)
    xnew = data['x'][index]
    ynew = data['y'][index]
    znew = data['z'][index]
    vxnew = data['vx'][index]
    vynew = data['vy'][index]
    vznew = data['vz'][index]
    massnew = data['m'][index]
    
    # Store positions and velocities of particles of given ptype from the COMP. 
    xnew2 = xnew - MergerCOMP[0]
    ynew2 = ynew - MergerCOMP[1]
    znew2 = znew - MergerCOMP[2]
    vxnew2 = vxnew - MergerCOMV[0]
    vynew2 = vynew - MergerCOMV[1]
    vznew2 = vznew - MergerCOMV[2]
    
    #read in distance component values and set units as kpc
    x = xnew2[particlenum]
    xcomp = float(x)*u.kpc
    y = ynew2[particlenum]
    ycomp = float(y)*u.kpc
    z = znew2[particlenum]
    zcomp = float(z)*u.kpc
    
    #calculate magnitude of distance squared in kpc
    threeDdist = np.sqrt(xcomp**2 + ycomp**2 + zcomp**2)
    
    #round final distance value to 3 decimal places
    rounded3Ddist = np.around(threeDdist, 3)
    
    #define km/s units
    kms = (u.km / u.s)
    
    #read in velocity component values and set units as km/s
    vx = vxnew2[particlenum]
    vxcomp = float(vx)*kms
    vy = vynew2[particlenum]
    vycomp = float(vy)*kms
    vz = vznew2[particlenum]
    vzcomp = float(vz)*kms
    
    #calculate magnitude of velocity
    threeDvel = np.sqrt(vxcomp**2 + vycomp**2 + vzcomp**2)
    
    #round final velocity value to 3 decimal places
    rounded3Dvel = np.around(threeDvel, 3)
    
    #return desired quantities
    return threeDdist.value, threeDvel.value

In [33]:
#create an array of distances and velocities for every particle in MW and M31
rmagsMW = np.zeros(len(MWx))
vmagsMW = np.zeros(len(MWx))
rmagsM31 = np.zeros(len(M31x))
vmagsM31 = np.zeros(len(M31x))

for i in range(len(MWx)):
    rmagsMW[i], vmagsMW[i] = ParticleInfo("MW_575.txt", 2, i)
    #rmagsM31[i], vmagsM31[i] = ParticleInfo("M31_575.txt", 2, i)

print(rmagsMW)

#this ran for more than 3 hours...

KeyboardInterrupt: 

In [ ]:
#create a function to calculate the mean velocities, and spread of velocities for stars for a given array of radii
r = np.arange(0.1, 30.0, 1.0)*u.kpc
#do I do this like calculating mass enclosed where I calculate the average velocity of particles enclosed in a certain radius???

def VelocityMeansandStds(r, v, rdata):
    #Inputs: 
        #r = array of set radii
        #v = array of velocity magnitudes
        #rdata = radii of stars
    #returns
        #array of average velocities
        #array of std of velocities
    meanvel = np.zeros(len(r))
    stdvel = np.zeros(len(r))
    
    #loop over radius array to define particles enclosed within radius
    for i in range(len(r)):
        #create index to select particles whose radii are less than the ith element of the radii array 
        index2 = np.where(rdata <= r[i])
            
        #make new array of radii and velocities that satisfy the above condition
        Rnew = rdata[index2]
        Vnew = v[index2]
            
        #store mean of all the velocities that fall under the condition as an element in meanvel
        meanvel[i] = np.mean(Vnew)
        stdvel[i] = np.std(Vnew)
        
    return meanvel, stdvel  
    

In [ ]:
#Use VelocityMeansandStds to calculate an array of meanvelocities and the standard deviation of velocities
#i.e. velocity dispersions for both the MW and M31 disk stars at this snapshot, then plot them as a function of radius 